In [1]:
import pandas as pd
import sqlite3, os, re

# Create DB for use later (no bugs here)

In [2]:
symptoms = pd.DataFrame([
    {"clinic": "Aurora St. Luke's Medical Center", "patient_id":"j808435", "description": "experiencing mild skin rashes. Redness lasted 4 days"},
    {"clinic": "Meriter Hospital", "patient_id":"a114506", "description":"itchy and red eyes. Suspected eye infection."},
    {"clinic": "Meriter Hospital", "patient_id":"d213306", "description":"Patient is experiencing a sore throat. Has had difficulties sleeping. temperature recorded at 99F since start of Monday"},
    {"clinic": "Aurora St. Luke's Medical Center", "patient_id":"g983akd", "description": "reported yeast infection on right foot sole."},
    {"clinic": "Meriter Hospital", "patient_id":"r572131", "description":"Lack of smell for past week. Appears asymptomatic, temperature at 100F"},
    {"clinic": "Meriter Hospital", "patient_id":"b083428", "description":"moderate fever for two nights and dizziness. reported 37.6C, last temperature 99.5F"},
    {"clinic": "Meriter Hospital", "patient_id":"p093485", "description":"mild fever. a week of phlegm buildup."},
    {"clinic": "Meriter Hospital", "patient_id":"i234000", "description":"experiencing light-headedness from walking. Weakness and lack of temperature regulation at night. Temperature is 99.95F"},
    {"clinic": "Meriter Hospital", "patient_id":"a332890", "description":" Started experiencing headaches 12 days ago. self-reported temperature of 101.5F. Immediate prescription provided"},
    {"clinic": "Meriter Hospital", "patient_id":"o506789", "description":"reported intense migraines throughout the day. mostly triggered by sunlight"},
    {"clinic": "Meriter Hospital", "patient_id":"i324810", "description":"temperature: 04/06/2021,103.5F,10:50. Sent to ER immediately"},
    {"clinic": "Meriter Hospital", "patient_id":"j345839", "description":"temperature: 04/06/2021,97.9F,11:23"},
])
contact = pd.DataFrame([
    {"patient_name": "Alex Joe", "city": "madison", "address": "123 Main St.", "number": "(555)434-6789", "patient_id": "a114506"},
    {"patient_name": "Tim John", "city": "madison", "address": "123 Main St.", "number": "(317)434-6767", "patient_id": "d213306"},
    {"patient_name": "John Smith", "city": "madison", "address": "123 Main St.", "number": "(728)890-9955", "patient_id": "-"},
    {"patient_name": "John Snow", "city": "madison", "address": "123 Main St.", "number": "(762)579-0043", "patient_id": "b083428"},
    {"patient_name": "Brian Gooligan", "city": "madison", "address": "123 Main St.", "number": "(348)189-0584", "patient_id": "p093485"},
    {"patient_name": "Jake Galliger", "city": "madison", "address": "123 Main St.", "number": "(555)485-2956", "patient_id": "i234000"},
    {"patient_name": "Austin Powers", "city": "madison", "address": "123 Main St.", "number": "-", "patient_id": "a332890"},
    {"patient_name": "Jack Yadav", "city": "madison", "address": "123 Main St.", "number": "(555)337-9834", "patient_id": "o506789"},
    {"patient_name": "Beth Sullivan", "city": "madison", "address": "123 Main St.", "number": "(555)227-3554", "patient_id": "i324810"},
    {"patient_name": "James Jameson", "city": "madison", "address": "123 Main St.", "number": "(555)908-2215", "patient_id": "j345839"},
    {"patient_name": "William Jeffrey", "city": "milwaukee", "address": "456 State St.", "number": "(442)743-8900", "patient_id": "j808435"},
    {"patient_name": "Walter White", "city": "milwaukee", "address": "456 State St.", "number": "(234)928-5719", "patient_id": "g983akd"},
])
connection = sqlite3.connect("info.db")
symptoms.to_sql("symptoms", connection, if_exists="replace", index=False)
contact.to_sql("contact", connection, if_exists="replace", index=False)
connection.close()

# Let's see what's in the DB (no bugs here)

In [3]:
assert os.path.exists("info.db")
connection = sqlite3.connect("info.db")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", connection)
print('Tables from database:', list(tables['name']))

Tables from database: ['symptoms', 'contact']


In [4]:
pd.read_sql("""
SELECT * FROM symptoms
""", connection)

,clinic,patient_id,description
0,Aurora St. Luke's Medical Center,j808435,experiencing mild skin rashes. Redness lasted ...
1,Meriter Hospital,a114506,itchy and red eyes. Suspected eye infection.
2,Meriter Hospital,d213306,Patient is experiencing a sore throat. Has had...
3,Aurora St. Luke's Medical Center,g983akd,reported yeast infection on right foot sole.
4,Meriter Hospital,r572131,Lack of smell for past week. Appears asymptoma...
5,Meriter Hospital,b083428,moderate fever for two nights and dizziness. r...
6,Meriter Hospital,p093485,mild fever. a week of phlegm buildup.
7,Meriter Hospital,i234000,experiencing light-headedness from walking. We...
8,Meriter Hospital,a332890,Started experiencing headaches 12 days ago. s...
9,Meriter Hospital,o506789,reported intense migraines throughout the day....


In [5]:
pd.read_sql("""
SELECT * FROM contact
""", connection)

,patient_name,city,address,number,patient_id
0,Alex Joe,madison,123 Main St.,(555)434-6789,a114506
1,Tim John,madison,123 Main St.,(317)434-6767,d213306
2,John Smith,madison,123 Main St.,(728)890-9955,-
3,John Snow,madison,123 Main St.,(762)579-0043,b083428
4,Brian Gooligan,madison,123 Main St.,(348)189-0584,p093485
5,Jake Galliger,madison,123 Main St.,(555)485-2956,i234000
6,Austin Powers,madison,123 Main St.,-,a332890
7,Jack Yadav,madison,123 Main St.,(555)337-9834,o506789
8,Beth Sullivan,madison,123 Main St.,(555)227-3554,i324810
9,James Jameson,madison,123 Main St.,(555)908-2215,j345839


# Print Contact Info for Madison Patients with Fevers

In [6]:
# GOAL: show contact info alongside symptoms for Madison patients

# BUG 1: Why does every row in contact pair with every row in symptoms, producing 12x12 result rows?
# BUG 2: Why are patients outside of Madison appearing here?
df = pd.read_sql("""
SELECT *
FROM contact INNER JOIN symptoms
""", connection)
df

,patient_name,city,address,number,patient_id,clinic,patient_id,description
0,Alex Joe,madison,123 Main St.,(555)434-6789,a114506,Aurora St. Luke's Medical Center,j808435,experiencing mild skin rashes. Redness lasted ...
1,Alex Joe,madison,123 Main St.,(555)434-6789,a114506,Meriter Hospital,a114506,itchy and red eyes. Suspected eye infection.
2,Alex Joe,madison,123 Main St.,(555)434-6789,a114506,Meriter Hospital,d213306,Patient is experiencing a sore throat. Has had...
3,Alex Joe,madison,123 Main St.,(555)434-6789,a114506,Aurora St. Luke's Medical Center,g983akd,reported yeast infection on right foot sole.
4,Alex Joe,madison,123 Main St.,(555)434-6789,a114506,Meriter Hospital,r572131,Lack of smell for past week. Appears asymptoma...
...,...,...,...,...,...,...,...,...
139,Walter White,milwaukee,456 State St.,(234)928-5719,g983akd,Meriter Hospital,i234000,experiencing light-headedness from walking. We...
140,Walter White,milwaukee,456 State St.,(234)928-5719,g983akd,Meriter Hospital,a332890,Started experiencing headaches 12 days ago. s...
141,Walter White,milwaukee,456 State St.,(234)928-5719,g983akd,Meriter Hospital,o506789,reported intense migraines throughout the day....
142,Walter White,milwaukee,456 State St.,(234)928-5719,g983akd,Meriter Hospital,i324810,"temperature: 04/06/2021,103.5F,10:50. Sent to ..."


In [7]:
# GOAL: print contact info for each fever mention (>=99.5F is a fever).
#       there should be 5 fevers and 2 non-fever temps...
# BUGS: several
for row in df.itertuples():
    print(f"{row.patient_name}: {row.description}")
    matches = re.findall(r"(\d{2}\.?\d?\d?)([F])", row.description)
    if len(matches) > 0:
        match = matches[0]
        temp, units = float(match[0]), match[1]
        if units == "C":
            temp = temp * 9/5 + 32
            units = "F"
        if float(temp) >= 99.5:
            print(f"    FEVER at {temp}{units}!  Call at {row.number}")
        else:
            print(f"    {temp}{units} is not a fever")

Alex Joe: experiencing mild skin rashes. Redness lasted 4 days
Alex Joe: itchy and red eyes. Suspected eye infection.
Alex Joe: Patient is experiencing a sore throat. Has had difficulties sleeping. temperature recorded at 99F since start of Monday
    99.0F is not a fever
Alex Joe: reported yeast infection on right foot sole.
Alex Joe: Lack of smell for past week. Appears asymptomatic, temperature at 100F
    FEVER at 100.0F!  Call at (555)434-6789
Alex Joe: moderate fever for two nights and dizziness. reported 37.6C, last temperature 99.5F
    FEVER at 99.5F!  Call at (555)434-6789
Alex Joe: mild fever. a week of phlegm buildup.
Alex Joe: experiencing light-headedness from walking. Weakness and lack of temperature regulation at night. Temperature is 99.95F
    FEVER at 99.95F!  Call at (555)434-6789
Alex Joe:  Started experiencing headaches 12 days ago. self-reported temperature of 101.5F. Immediate prescription provided
    1.5F is not a fever
Alex Joe: reported intense migraines thr

In [8]:
connection.close()